# Using the local password manager

The package `psypwdm` is a local password manager serving to locally, securely and conveniently store passwords to personal systems.

The password manager is intended to be used with the relational database PostgreSQL (which can be downloaded [here](https://www.postgresql.org/)), so please ensure this is installed.

In this tutorial we will walk through the process required to initialise the password manager table in the postgres databse, how to insert and retrieve entries and how to delete the table. 

**Note** *The password itself will not be stored, only its `SHA256` hash, salted appropriately. This means if you forget a password, you cannot simply read it off from a stored entry, however you can try to guess what the password is through the `retrieve_for_pwd` function discussed later.* 

## Installation

Installing the password manager `psypwdm` is straightforward. It can be pip installed with the following command:

    $ pip install psypwdm

With PostgreSQL up and running, simply importing `psypwdm` will create the password manager table as follows.

In [1]:
import psypwdm as pwdm

PasswordManager has been initialised


The printed statement above will only appear if the `passwordmanager` table is created for the first time on import.

**Warning.** *Be advised that on import, `psypwdm` will create a table name `passwordmanager`. If this table already exists, then nothing will happen. Method calls on `psypwdm` will modify this table. This should be noted if you have a `passwordmanager` table already initialised for other reasons.*

## Adding entries

The insert function, viewed as a method call on `pwdm`, enables inserting entries into the `passwordmanager` table. 

The codeblock below demonstrates adding the following entries.

```
1.

useraccountsystemtype: gmail account

username: example@gmail.com

password: examplepassword

comments: password hint, example of a password

```

```
2.

useraccountsystemtype: bank account

username: bankman

password: givememymoney!

comments: password hint, what do i want?

```

In [2]:
pwdm.insert(
    useraccountsystemtype='gmail account',
    username='example@gmail.com',
    comments='password hint, example of a password'    
)

insert password:  ········


entry successfully added to passwordmanager!


In [3]:
pwdm.insert(
    useraccountsystemtype='bank account',
    username='bankman',
    comments='password hint, what do i want?' 
)

insert password:  ········


entry successfully added to passwordmanager!


**Note.** *All fields in the password manager are stored as plain text except for the password.*

## Retrieval

The `retrieve` function allows for retrieval of entries in the `passwordmanager` table. Entries can be retrieved all at once, by specified plain text value or by password, if this is known. Each retrieval method will be demonstrated below.

**Note.** *Since passwords are not stored in plain text, only hash collision matches will be returned.*

### Retrieve all 

To retrieve *all* entries, simply call `retrieve` without any arguments.

In [4]:
pwdm.retrieve()

[(1,
  '2024-07-20 08:14:28.564229',
  'gmail account',
  'example@gmail.com',
  'cc08af454351e59ed90692cd52f9d1a92519a4a87f55d9a42522588c6a6a877f',
  '$2b$12$6KFYf4sP/nO1lvriG7Qh8e                                   ',
  'password hint, example of a password'),
 (2,
  '2024-07-20 08:14:36.152516',
  'bank account',
  'bankman',
  '5513775b054f7fec9d38d741593841db0a75fd3b5696adce9dc270d2b78850fc',
  '$2b$12$nqBq5725PCJBAo7KXd6eQ.                                   ',
  'password hint, what do i want?')]

Observe that there are more fields than those for direct user input, explained as follows:

- the first is a generic ID field
- the second is a timestamp, recorded the time when the entry was created. This is useful if changes to the same system has been made (such as a password change), so that only the most recent entry wil be relevant.
- the third is the useraccountsystemtype, passed on input
- the fourth is the username, passed on input
- the fifth is the hash of the password entered on input
- the sixth is the salt value used in hashing, ensuring that if the same password is used across multiple accounts, this will fact itself will not be readily visible
- the seventh is the comments pertaining to the entry, passed on input

### Retrieve by field

For any field other than `password`, entries can be returned based on that field value. This is implemented through passing the field name(s) and value(s) of to be retrieved, demonstrated below.


In [5]:
pwdm.retrieve(useraccountsystemtype='bank account')

[(2,
  '2024-07-20 08:14:36.152516',
  'bank account',
  'bankman',
  '5513775b054f7fec9d38d741593841db0a75fd3b5696adce9dc270d2b78850fc',
  '$2b$12$nqBq5725PCJBAo7KXd6eQ.                                   ',
  'password hint, what do i want?')]

In [6]:
pwdm.retrieve(useraccountsystemtype='bank account', username='banksy')

[]

We see above that while there exists an entry for the `useraccountsystemtype` is `bank account`, there are no entries for when the `username` field is also `banksy`. That is, the `passwordmanager` table has no record of a `banksy` user associated with a `bank account`.

### Retrieve by password

As demonstrated above, passwords are not stored. Only their hash + salt is stored. Since the salt is in plain text, it can be used in conjunction with any password guess to check if the password guessed matches the password of any entry in the `passwordmanager` table.

This functionality is implemented through the `retrieve_for_pwd` function, demonstrated below. Note, *no* arguments are passed to the function. It will request a password securely through Python's built-in `getpass` library.


In [7]:
pwdm.retrieve_for_pwd()

insert password:  ········


[(1,
  '2024-07-20 08:14:28.564229',
  'gmail account',
  'example@gmail.com',
  'cc08af454351e59ed90692cd52f9d1a92519a4a87f55d9a42522588c6a6a877f',
  '$2b$12$6KFYf4sP/nO1lvriG7Qh8e                                   ',
  'password hint, example of a password')]

In [8]:
pwdm.retrieve_for_pwd()

insert password:  ········


[(2,
  '2024-07-20 08:14:36.152516',
  'bank account',
  'bankman',
  '5513775b054f7fec9d38d741593841db0a75fd3b5696adce9dc270d2b78850fc',
  '$2b$12$nqBq5725PCJBAo7KXd6eQ.                                   ',
  'password hint, what do i want?')]

If we guessed incorrectly, the nothing will be returned as seen below.

In [9]:
pwdm.retrieve_for_pwd()

insert password:  ········


[]

## Output

We might like to write the contents of the `passwordmanager` table to a more universal file format for analysis by other means. This is the purpose of the `out` function. It passes a file path to write the table, relative to the current directory. 

Below, the contents of `passwordmanager` will be written to the file `out.csv`.


In [10]:
pwdm.out('out.csv')

Success! Contents of passwordmanager have been written to out.csv


## Delete

Finally, if we are done with our password manager and/or wish to start fresh with a new table, the current table can be deleted with the `delete` function. This function passes no parameters, however it will require confirmation before deleting.

In [11]:
pwdm.delete()

Are you sure you want to delete the passwordmanager table and contents? [Y/N] Y


the passwordmanager table has been deleted.
